In [ ]:
# from getonbrd import *
import pandas as pd

In [2]:
from bson.objectid import ObjectId
from pymongo import MongoClient

# Manage database.

In [69]:
client = MongoClient('mongodb://localhost:27017/')
my_db = client['getonbrd_job_offers']

In [70]:
collections = my_db.list_collection_names()
collections

['jobs_sysadmin_devops_qa',
 'jobs_mobile_developer',
 'jobs_programming',
 'jobs_data_science_analytics',
 'jobs_machine_learning_ai']

## Count_documents().

In [71]:
for c in collections:
    print(f"{my_db[c].count_documents({})} : {c}")

103 : jobs_sysadmin_devops_qa
40 : jobs_mobile_developer
454 : jobs_programming
21 : jobs_data_science_analytics
1 : jobs_machine_learning_ai


## Newest entry. "[published_at]"

In [77]:
collection = my_db[collections[2]]

r = collection.aggregate([
    {'$project':{
                '_id':0,
                'title':1,
                'published_at':1,
                'seniority':1,
                }},
    
    {'$sort':{'published_at':-1}},
    {'$limit':1},
])

next(r)

{'title': 'Developer Engineer for Fintech Squad',
 'published_at': datetime.datetime(2022, 8, 23, 17, 51, 23),
 'seniority': 'semi_senior'}

## n Jobs

In [112]:
collection = my_db['jobs_programming']

enejobs = collection.aggregate([
    {'$project':{
                '_id':0,
                'title':1,
                'published_at':1,
                'seniority':1,
                'public_url':1,
                'tags':1,
                'company':1,
                }
    },
    {'$sort':{'published_at':-1}},
    {'$limit':1},
])
for i in enejobs:
    print(f"\n{i.get('published_at')} // {i.get('title')} // Company: {i['company']} // {i.get('seniority')} // {i.get('public_url') }")
    # if i['seniority']==('junior' or 'sin_experiencia'):
        # if ( 'matlab' in i['tags']):
            # print(i['tags'][:5])
            # print(f"\n{i.get('published_at')} // {i.get('title')} // {i.get('seniority')} // {i.get('public_url') }")


2022-08-23 17:51:23 // Developer Engineer for Fintech Squad // Company: wherEX // semi_senior // https://www.getonbrd.com/jobs/fullstack-developer-semi-senior-wherex-remote


## LOOP, newest entry.

In [74]:
categories = [
    'jobs_machine_learning_ai',
    'jobs_data_science_analytics',
    'jobs_mobile_developer',
    'jobs_sysadmin_devops_qa',
    'jobs_programming', 
]
for c in categories:
    collection = my_db[c]
    r = collection.aggregate([
        {'$project':{
                    '_id':0,
                    'published_at':1,
                    'company':1,
                    'seniority':1,
                    # 'title':1,
                    # 'tags':1,
                    }},
        
        {'$sort':{'published_at':-1}},
        {'$limit':1},
    ])
    print(f"{next(r)} \t CATEGORY {c}")

{'published_at': datetime.datetime(2022, 2, 7, 12, 6, 48), 'seniority': 'semi_senior', 'company': 11760} 	 CATEGORY jobs_machine_learning_ai
{'published_at': datetime.datetime(2022, 8, 23, 8, 33, 34), 'seniority': 'semi_senior', 'company': 2314} 	 CATEGORY jobs_data_science_analytics
{'published_at': datetime.datetime(2022, 8, 23, 10, 34, 10), 'seniority': 'semi_senior', 'company': 3653} 	 CATEGORY jobs_mobile_developer
{'published_at': datetime.datetime(2022, 8, 23, 13, 22, 42), 'seniority': 'senior', 'company': 7738} 	 CATEGORY jobs_sysadmin_devops_qa
{'published_at': datetime.datetime(2022, 8, 23, 17, 51, 23), 'seniority': 'semi_senior', 'company': 'wherEX'} 	 CATEGORY jobs_programming


# Query database. Aggregation.

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
my_db = client['getonbrd_api']

In [ ]:
categories = [
    'jobs_programming',
    'jobs_data_science_analytics',
    'jobs_sysadmin_devops_qa',
    'jobs_machine_learning_ai',
    'jobs_mobile_developer',
]

In [ ]:
colls = my_db[categories[0]]
colls

In [ ]:
cursor_mdb = colls.aggregate([{'$project':{'_id':False}}])

trabajos = []
for doc in cursor_mdb:
    trabajos.append(doc)

In [ ]:
df = pd.DataFrame(trabajos)
df.set_index('published_at', inplace=True)

df.columns

In [ ]:
df.loc['2022']['seniority'].value_counts().plot(kind='bar')

# Request new jobs from api

In [53]:
import requests
from datetime import datetime

In [54]:
baseurl = 'https://www.getonbrd.com/api/v0/'

def _request(url):
    payload = {}
    headers = {}
    try:
        response = requests.request("GET", url, headers=headers, data=payload)
        if response.status_code==200:
            return response.json()
    except:
        return response.status_code

In [59]:
# PARAMS
per_page = 5
page = 1
# category = 'programming'
# category = 'sysadmin-devops-qa'
category = 'mobile-developer'

endpoint = f'categories/{category}/jobs?per_page={per_page}&page={page}&expand=["company"]'
url = baseurl+endpoint

new_jobs = _request(url)

new_jobs.keys()

dict_keys(['data', 'meta'])

# Explore Tags

In [113]:
import pickle
import operator

In [114]:
with open('data/list_programming_2022-08-23 18:00:54.210801.pckl', 'rb') as bindata:
    jobs_prog = pickle.load(bindata)

len(jobs_prog)

In [119]:
# All tags, aprox. 10 * job publication.

all_tags = []
for j in jobs_prog:
    all_tags += j.get('tags')

len(all_tags)

4490

In [139]:
# Unique tags isolated.

unique_tags = list(set(all_tags.copy()))
len(unique_tags)

176

## Count tags frequencies.

In [ ]:
[
('remote-handbook', 348), ('matlab', 116), ('business-intelligence', 116), ('cardano', 108),
('data-science', 79),  ('etl', 53), ('english', 43), ('angular', 41),
('continuous-integration', 31), ('data-visualization', 29),  ('analytics', 21),
('deep-learning', 21), ('data-analysis', 17), ('git', 14),
]

In [ ]:
tags_dict = dict(zip(unique_tags, [0 for x in range(len(unique_tags))] ))

In [146]:
for t in all_tags:
    tags_dict[t] += 1

In [196]:
tagx = sorted(tags_dict.items(), key=operator.itemgetter(1))
tagx.reverse()

In [208]:
tagx[:25]

[('remote-handbook', 348),
 (None, 316),
 ('abap', 262),
 ('bootstrap', 146),
 ('director-de-cuentas', 130),
 ('mocha', 126),
 ('chatbot', 119),
 ('matlab', 116),
 ('business-intelligence', 116),
 ('drupal', 115),
 ('planificacion-de-medios', 111),
 ('cardano', 108),
 ('motion-graphics', 106),
 ('access-point', 103),
 ('animation', 100),
 ('awesome-companies', 99),
 ('elm', 94),
 ('amqp', 82),
 ('data-science', 79),
 ('ampl', 75),
 ('brightscript', 66),
 ('gis', 63),
 ('ext-js', 63),
 ('osb', 61),
 ('branding', 57)]